In [37]:
import pandas as pd
from pathlib import Path

In [38]:
BASE_DIR = '../../data/rawdata/'

In [ ]:
def read_bh(file):
    '''Auxiliary function to read the data from the behavioural file
    in the specified format and with meaningful column names.'''
    df = pd.read_csv(file, header=None,
        names=['o', 'ab', 'show_times', 'stim', 'PER', 'big_small', 'nl', 'onset', 'opt', 'delay']).drop(
        columns=['o', 'PER']
    )
    df['opt'] = df['opt'].astype('Int64')

    return df

In [43]:
# Get all subjects' directories
base_path = Path(BASE_DIR)
subjects = sorted([d for d in base_path.iterdir() if d.is_dir()])
data = {}

# Load all behaviour files
for subject in subjects:
    # Behaviour files are CSVs in the beh directory
    beh_files = sorted(subject.glob('beh/*.csv'))
    csvs = {}
    # Each subject should have 2 behaviour files
    if len(beh_files) != 2:
        print(f'Error: {subject.name} does not have 2 behaviour files')
        continue
    # Load the CSVs with pandas.read_csv
    csvs['rep1'] = read_bh(beh_files[0])
    csvs['rep2'] = read_bh(beh_files[1])
    data[subject.name] = csvs

In [40]:
print(data.keys())

dict_keys(['o01', 'o02', 'o03', 'o04', 'o05', 'o06', 'o07', 'o08', 'o09', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'o17', 'o18', 'y01', 'y02', 'y03', 'y04', 'y05', 'y06', 'y07', 'y08', 'y09', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19'])


In [44]:
data['y01']['rep1'].head()

,ab,show_times,stim,big_small,nl,onset,opt,delay
0,A,3,stim/faint.bmp,S,NL,29988,4,807
1,A,3,stim/toilett.bmp,B,NL,32988,3,636
2,D,1,stim/flashlight.bmp,S,NL,35988,4,667
3,A,3,stim/broccoli.bmp,S,L,38988,4,550
4,A,3,stim/car.bmp,B,NL,41988,3,585


In [45]:
data['y01']['rep2'].head()

,ab,show_times,stim,big_small,nl,onset,opt,delay
0,D,1,stim/peach.bmp,S,L,29972,4,601
1,A,3,stim/baby.bmp,B,L,36972,3,613
2,A,3,stim/orange.bmp,S,L,39972,4,598
3,A,3,stim/orange.bmp,S,L,42973,4,458
4,A,3,stim/cruiseship.bmp,B,NL,45972,3,578
